# Agent Tools

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
brave_api_key = os.getenv("BRAVE_API_KEY")

In [2]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_openai import ChatOpenAI

In [3]:
# For tools/agents, use temeprature=0 to get deterministic results
llm = ChatOpenAI(temperature=0)

In [4]:
# Tools:
# # https://python.langchain.com/v0.2/docs/integrations/tools/
# For most of them we need to set an API key
# Check each integration page for instructions
# Here the SerpAPI is used: Search Engine Results API
# https://python.langchain.com/v0.2/docs/integrations/tools/serpapi/
# https://serpapi.com/
# from langchain_community.utilities import SerpAPIWrapper
# However, I decided to choose the Brave Search API
# https://python.langchain.com/v0.2/docs/integrations/tools/brave_search/
# https://api.search.brave.com
from langchain_community.tools import BraveSearch

In [5]:
search_tool = BraveSearch.from_api_key(
    api_key=brave_api_key,
    search_kwargs={"count": 3}
)

In [6]:
type(search_tool) # langchain_community.tools.brave_search.tool.BraveSearch

langchain_community.tools.brave_search.tool.BraveSearch

In [7]:
search_tool.run("obama middle name") # [{"title": "Barack Obama - Wikipedia", "link": "https://en.wikipedia.org/wiki/Barack_Obama", "snippet": "In January 2007, he left...

'[{"title": "Barack Obama - Wikipedia", "link": "https://en.wikipedia.org/wiki/Barack_Obama", "snippet": "In January 2007, he left the ... subcommittee on European Affairs. As a member of the Senate Foreign Relations Committee, <strong>Obama</strong> made official trips to Eastern Europe, the <strong>Middle</strong> East, Central Asia, and Africa...."}, {"title": "Obama\'s Middle Name - My Last Name - is \'Hussein.\' So? -", "link": "https://www.cair.com/cair_in_the_news/obamas-middle-name-my-last-name-is-hussein-so/", "snippet": "Many Americans understand that common names don\\u2019t only come in the form of a \\u201cSmith\\u201d or a \\u201cJohnson.\\u201d Perhaps, they have a neighbor, mechanic or teacher named Hussein. Or maybe they\\u2019ve seen fashion designer Hussein Chalayan in the pages of Vogue or recall <strong>King Hussein</strong>, our ally in ..."}, {"title": "What\'s up with Obama\'s middle name? - Quora", "link": "https://www.quora.com/Whats-up-with-Obamas-middle-name

In [8]:
# We load the LC math tool and add the Brave search tool to the list
tools = load_tools(["llm-math"], llm=llm) # list
tools = tools + [search_tool]

In [9]:
# We create a Zero-shot Agent
# WARNING: initialize_agent is deprecated...
# Instead, we should use new agent constructor methods:
# create_react_agent, create_json_agent, create_structured_chat_agent, ...
agent = initialize_agent(
    tools, # Brave-Search
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True # check what is happening inside
)

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent.run("What is the year Albert Einstein was born? What is this year multiplied by 5?")

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I can use a calculator to find the year Albert Einstein was born and then multiply it by 5.
Action: Calculator
Action Input: 2022 - 142
Observation: Answer: 1880
Thought:Now I can multiply 1880 by 5 to get the final answer.
Action: Calculator
Action Input: 1880 * 5
Observation: Answer: 9400
Thought:I now know the final answer.
Final Answer: 9400

> Finished chain.


'9400'

In [11]:
# Same agent but using the new interface
# https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html#langchain.agents.react.agent.create_react_agent
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor

In [12]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [13]:
agent = create_react_agent(
    llm,
    tools,
    prompt
)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [15]:
agent_executor.invoke({"input": "What is the year Albert Einstein was born? What is this year multiplied by 5?"})
# {'input': 'What is the year Albert Einstein was born? What is this year multiplied by 5?',
# 'output': 'Albert Einstein was born in 1879. 1879 multiplied by 5 is 9395.'}

{'input': 'What is the year Albert Einstein was born? What is this year multiplied by 5?',
 'output': 'Albert Einstein was born in 1879. 1879 multiplied by 5 is 9395.'}

### Python Agent Tool

This agent can run Python code. It is very powerful, but can be dangerous at the same time.

In [26]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent

In [27]:
llm = ChatOpenAI(temperature=0.0)

In [29]:
agent = create_python_agent(
    tool=PythonREPLTool(),
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [30]:
python_list = [1,1,2,3,1,2,5,50,1,4,8,6,78,9]

In [31]:
agent.run(f"Sort this list: {python_list}") # '[1, 1, 1, 1, 2, 2, 3, 4, 5, 6, 8, 9, 50, 78]'



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We can use the `sorted()` function in Python to sort the list.
Action: Python_REPL
Action Input: sorted([1, 1, 2, 3, 1, 2, 5, 50, 1, 4, 8, 6, 78, 9])
Observation: 
Thought:The list has been sorted in ascending order.
Final Answer: [1, 1, 1, 1, 2, 2, 3, 4, 5, 6, 8, 9, 50, 78]

> Finished chain.


'[1, 1, 1, 1, 2, 2, 3, 4, 5, 6, 8, 9, 50, 78]'